This notebook uses, a priori knowledge on the question and user to guess the outcome.

The idea is to gather user stats under dictionary variables, such as user_success,user_tags, and user_parts
And also general stats on the question such as, question_success, all_tags and all_parts.

And use these stats to predict the outcome.
So basically, if the user has given success values (overall, parts and tags success for the part/tags of the question)...
And the question has its own statistics, (overall question success and, general parts and tags success for the part/tags of the question)...
Then the a valid prediction is a function of these values.



In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)




In [ ]:
#read in all the pickled data you need
import pickle

with open('../input/settim/question_success.pickle', 'rb') as handle:
    question_success = pickle.load(handle)
    
with open('../input/settim/user_success.pickle', 'rb') as handle:
    user_success = pickle.load(handle)
    
with open('../input/settim/all_parts.pickle', 'rb') as handle:
    all_parts = pickle.load(handle)
    
with open('../input/settim/all_tags.pickle', 'rb') as handle:
    all_tags = pickle.load(handle)
    
with open('../input/settim/user_parts.pickle', 'rb') as handle:
    user_parts = pickle.load(handle)
    
with open('../input/settim/user_tags.pickle', 'rb') as handle:
    user_tags = pickle.load(handle)

In [ ]:
#we need the names (user_id) for the existing users to pull the stats, if that is a user_id for which 
#we have the stats...
names=pd.read_csv('../input/settim/userids.csv',usecols=[1])
names=names['0']

In [ ]:
#we need to pull out the parts and tags related to each question that need be predicted.
#q will be a simple dataframe of questions and their associated part and tags...

dtypes={'part':'int8','question_id':'int16'}
q=pd.read_csv('../input/riiid-test-answer-prediction/questions.csv',usecols=[0,3,4]) #this is the original questions if need to check...

q.loc[10033,'tags']='-1'   #that is for the one question without a tag...
tags_list = [list(map(int,x.split())) for x in q.tags.values]   #now this makes a list of tags in q,, to be used for taganalysis
q['tags'] = tags_list  #now tags are nothing but a list of integers
q=q.rename(columns={'question_id':'content_id'})


In [ ]:
def riidpredictor(dataframe):             
    #this predictor iterates over all the lines of the test data, one by one,
    #pulls the stats of the user if the user_id within names
    #and takes a simple median as a prediction...
    
    #watch, how the results list is being populated, which will include the available prediction canditates
    #each prediction candidate, is a function of the known stats, of the question and its tags, and parts
    #as well as the users success.
    
    
    for row in dataframe.itertuples(): #iterates over rows of test dataframe

        tags=q.loc[row.content_id]['tags']  #pulls out relevant tags for the question of current row
        part=q.loc[row.content_id]['part']  #pulls out the parts
        user=int(row.user_id)               #pulss out the user
        results=[]                          #initializes the results list

        q_success=question_success['mean'][row.content_id]              #this is the simple question success
        
        if user not in names.values:                       #check if the user in existing names
            results.append(q_success)                      #if not, add existing simple question_success to the list
        else: 
            initial_success=user_success['mean'][user]     # if user already in the names
            genel_success=q_success + (initial_success - .626)  #project general question_success to the user_succes
                                           #main idea: if the user is xx% above the overall success rate, than the user
                                           # will be xx% above question_success (.626 is a close figure to overall success rate)
            results.append(genel_success)  #use this stats in the results list.

            if (user,part) in user_parts['mean'].keys():  #check if the user has already answered question in this part
                uparts=user_parts['mean'][(user,part)]    # if yes, use user's part success
                results.append(uparts)

                aparts=all_parts['mean'][part]              #this is the general parts success
                aparts_touser=q_success + (uparts - aparts) #here the idea is to produce another metric for the prediction, namely:
                                                            #if the user is xx%above overall parts'success, the she/he will be 
                                                            # xx% above overall question success
                
                results.append(aparts_touser)               #add this also the the prediction candidates...
                            #at this point we have 3 numbers in the results (if user in already seen names)

            if tags[0]!=-1:                                 #a simple check for the content_id: 10033
                for tag in tags:                            #which I have tagged with -1 (I know, not very efficient..
                                                            #to check this in every iteration but, here it is//)
                        
                    if (user,tag) in user_tags['mean'].keys():  #again the idea is, if there is a user,tag success
                        utag=user_tags['mean'][(user,tag)]      #use it in your predictions
                        results.append(utag)

                        atag=all_tags['mean'][tag]              #also with the same analysis of above as in parts
                        atag_touser=q_success + (utag - atag)   #if the user is more successful in that certain tag of the question
                                                                #the tag predictor says, she/he will be that amount better
                                                                #in that specific question
                        results.append(atag_touser)



        prediction= (q_success + np.median(results)) / 2   #one thing that is observed: question success has less variation
                                                           #than user success values...
                                                           #this averaging emphasizes the question success
        prediction=min(max(prediction,0.01),0.99)       #revised calculations for tags and parts above, make it possible to
                                                        #see values sucs as 1,001 or -0.001
                                                        #this line squeezes all within prediction limits...

        dataframe.loc[row.Index,'answered_correctly']=prediction  #AND THAT IS IT....

        
    return dataframe


In [ ]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()


In [ ]:
prior_test_df = None

In [ ]:
for (test_df, sample_prediction_df) in iter_test:  #MIGHT BE BETTER TO CHECK FIRST THE riidpredictor function above
                                                   #merge the prior test frame with questinons to add part and tags for each
                                                   #question...
                                                   #using prior test frame to update stats, we will ise in the riidpredictor
                
    if (prior_test_df is not None): 
        
        prior_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        del prior_test_df['prior_group_answers_correct']
        

    
        prior_test_df=prior_test_df.reset_index().merge(q, how="left",on='content_id')
        
        
        for line in prior_test_df.itertuples():
            if line.content_type_id!=0: 
                continue
                
            #for question_success:
            if line.content_id in question_success['mean']: 
                question_success['mean'][line.content_id]=(question_success['count'][line.content_id] * question_success['mean'][line.content_id] + line.answered_correctly)/(question_success['count'][line.content_id]+1)
                question_success['count'][line.content_id]+=1
            else:
                question_success['mean'][line.content_id]=line.answered_correctly
                question_success['count'][line.content_id]=1

                
            #for user_success:
            if line.user_id in user_success['mean']:
                user_success['mean'][line.user_id]=(user_success['count'][line.user_id] * user_success['mean'][line.user_id] + line.answered_correctly)/(user_success['count'][line.user_id]+1)
                user_success['count'][line.user_id]+=1
            else:
                user_success['mean'][line.user_id]=line.answered_correctly
                user_success['count'][line.user_id]=1
                
            
            #for user_parts:
            if (line.user_id,line.part) in user_parts['mean']:
                user_parts['mean'][(line.user_id,line.part)]=(user_parts['count'][(line.user_id,line.part)] * user_parts['mean'][(line.user_id,line.part)] + line.answered_correctly)/(user_parts['count'][(line.user_id,line.part)]+1)
                user_parts['count'][(line.user_id,line.part)]+=1
            else:
                user_parts['mean'][(line.user_id,line.part)]=line.answered_correctly
                user_parts['count'][(line.user_id,line.part)]=1
        
            #for all_parts:
            if line.part in all_parts['mean']:
                all_parts['mean'][line.part]=(all_parts['count'][line.part] * all_parts['mean'][line.part] + line.answered_correctly)/(all_parts['count'][line.part]+1)
                all_parts['count'][(line.part)]+=1
            else:
                all_parts['mean'][line.part]=line.answered_correctly
                all_parts['count'][line.part]=1
                
                
            #tags analysis
            if line.tags[0]!=-1:
                for tag in line.tags: 
                    
                    #for user_tags:
                    if (line.user_id,tag) in user_tags['mean']:
                        user_tags['mean'][(line.user_id,tag)]=(user_tags['count'][(line.user_id,tag)] * user_tags['mean'][(line.user_id,tag)] + line.answered_correctly)/(user_tags['count'][(line.user_id,tag)]+1)
                        user_tags['count'][(line.user_id,tag)]+=1
                    else:
                        user_tags['mean'][(line.user_id,tag)]=line.answered_correctly
                        user_tags['count'][(line.user_id,tag)]=1
                        
                    #for all_tags:
                    if tag in all_tags['mean']:
                        all_tags['mean'][tag]=(all_tags['count'][tag] * all_tags['mean'][tag] + line.answered_correctly)/(all_tags['count'][tag]+1)
                        all_tags['count'][tag]+=1
                    else:
                        all_tags['mean'][tag]=line.answered_correctly
                        all_tags['count'][tag]=1
                        
            #also we need to update names
            if line.user_id not in names.values:
                names.loc[names.index.max()+1] =line.user_id
        
    
    prior_test_df = test_df[['row_id','user_id','content_id','content_type_id','prior_group_answers_correct']].copy()
    
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    results=riidpredictor(test_df)
    env.predict(results[['row_id', 'answered_correctly']])